In [26]:
from unittest.mock import patch

### Emotional journal tests

In [27]:
from web.app.assistant.emotional_journal import *

In [28]:
# New journal update
journal = '{}'
updates_count = 0
gpt_model = ''

journal = EmotionalJournal(journal, updates_count, gpt_model)

mock_response = '{"emotion": 100}'
chat = ['Hi', 'How can I assist']

with patch('web.app.assistant.emotional_journal.openai_chat_request', return_value=[mock_response, 'token usage']):
    result = await journal.update_journal(chat)
    print("Result: ", result)
    assert result == ({"emotion": 100}, 1, "token usage")

____________________
--- Journal prompt:
 Conversation:|
Hi
How can I assist| 
--- Journal update response:
 {"emotion": 100}
____________________
Result:  ({'emotion': 100}, 1, 'token usage')


In [29]:
#Update of exisiting journal with new and exsisting emotion
 
journal = '{"emotion": 100}'
updates_count = 1
 
gpt_model = ''

journal = EmotionalJournal(journal, updates_count,   gpt_model)

mock_response = '{"emotion": 50, "emotion2": 50}'
chat = ['Hi']
    
with patch('web.app.assistant.emotional_journal.openai_chat_request', return_value=[mock_response, 'token usage']):
    result = await journal.update_journal(chat)
    print("Result: ", result)
    # e1 = (100 + 50) / 2 = 75
    # e2 = 50 / 2 = 25
    assert result == ({"emotion": 75, "emotion2": 25}, 2, 'token usage')


____________________
--- Journal prompt:
 Conversation:|
Hi| 
--- Journal update response:
 {"emotion": 50, "emotion2": 50}
____________________
Result:  ({'emotion': 75.0, 'emotion2': 25.0}, 2, 'token usage')


In [30]:
#Empty response from GPT
 
journal = '{}'
updates_count = 0
 
gpt_model = ''

journal = EmotionalJournal(journal, updates_count,   gpt_model)

mock_response = '{}'
chat = ['Hi']
    
with patch('web.app.assistant.emotional_journal.openai_chat_request', return_value=[mock_response, 'token usage']):
    result = await journal.update_journal(chat)
    print("Result: ", result)
    assert result == (None, None, 'token usage')

____________________
--- Journal prompt:
 Conversation:|
Hi| 
--- Journal update response:
 {}
____________________
Result:  (None, None, 'token usage')


In [31]:
# Multiple updates of existing journal
 
journal = '{"e": 10, "e2": 15, "e3": 62.5}'
updates_count = 2

gpt_model = ''

journal = EmotionalJournal(journal, updates_count,   gpt_model)

mock_response = '{"e": 30, "e4": 5}'
chat = ['Hi']
    
with patch('web.app.assistant.emotional_journal.openai_chat_request', return_value=[mock_response, 'token usage']):
    result = await journal.update_journal(chat)
    print("1. Result: ", result)
    journal, updates_count, token_usege = result

mock_response = '{"e2": 20}'
with patch('web.app.assistant.emotional_journal.openai_chat_request', return_value=[mock_response, 'token usage']): 
    journal = EmotionalJournal(json.dumps(journal), updates_count,   gpt_model)
    result = await journal.update_journal(chat)
    print("2. Result: ", result)
    journal, updates_count, token_usege = result
    
mock_response = '{"e5": 100}'
with patch('web.app.assistant.emotional_journal.openai_chat_request', return_value=[mock_response, 'token usage']): 
    journal = EmotionalJournal(json.dumps(journal), updates_count,   gpt_model)
    result = await journal.update_journal(chat)
    print("3. Result: ", result)
    
# e = (10*2 + 30) / 5 = 10
# e2 (15*2 + 10) / 5 = 10
# e3 (60*2) / 5 = 25
# e4 5/5
# e5 100 /5
assert result == ({'e': 10.0, 'e2': 10.0, 'e3': 25.0, 'e4': 1.0, 'e5': 20.0}, 5, 'token usage')

____________________
--- Journal prompt:
 Conversation:|
Hi| 
--- Journal update response:
 {"e": 30, "e4": 5}
____________________
1. Result:  ({'e': 16.67, 'e2': 10.0, 'e3': 41.67, 'e4': 1.67}, 3, 'token usage')
____________________
--- Journal prompt:
 Conversation:|
Hi| 
--- Journal update response:
 {"e2": 20}
____________________
2. Result:  ({'e': 12.5, 'e2': 12.5, 'e3': 31.25, 'e4': 1.25}, 4, 'token usage')
____________________
--- Journal prompt:
 Conversation:|
Hi| 
--- Journal update response:
 {"e5": 100}
____________________
3. Result:  ({'e': 10.0, 'e2': 10.0, 'e3': 25.0, 'e4': 1.0, 'e5': 20.0}, 5, 'token usage')


In [32]:
# Update journal with 1 invalid value in response
 
journal = '{"e": 50}'
updates_count = 1
 
gpt_model = ''

Journal = EmotionalJournal(journal, updates_count,   gpt_model)

mock_response = '{"e": -1, "e1": 10}'
chat = ['Hi', 'How can I assist']

with patch('web.app.assistant.emotional_journal.openai_chat_request', return_value=[mock_response, 'token usage']):
    result = await Journal.update_journal(chat)
    print("Result: ", result)
    assert result == ({'e': 25.0, 'e1': 5.0}, 2, 'token usage')

____________________
--- Journal prompt:
 Conversation:|
Hi
How can I assist| 
--- Journal update response:
 {"e": -1, "e1": 10}
____________________
Result:  ({'e': 25.0, 'e1': 5.0}, 2, 'token usage')


In [33]:
# Update journal with all invalid values in response
 
journal = '{"e": 10, "e2": "20"}'
updates_count = 3
 
gpt_model = ''

Journal = EmotionalJournal(journal, updates_count,   gpt_model)

mock_response = '{"e": -1, "e1": 101, "e3": 100.1, "2e": -0.1}'
chat = ['Hi', 'How can I assist']

with patch('web.app.assistant.emotional_journal.openai_chat_request', return_value=[mock_response, 'token usage']):
    result = await Journal.update_journal(chat)
    print("Result: ", result)
    assert result == (None, None, 'token usage')

____________________
--- Journal prompt:
 Conversation:|
Hi
How can I assist| 
--- Journal update response:
 {"e": -1, "e1": 101, "e3": 100.1, "2e": -0.1}
____________________
Result:  (None, None, 'token usage')


In [34]:
# None in response
 
journal = '{}'
updates_count = 0
 
gpt_model = ''

journal = EmotionalJournal(journal, updates_count,   gpt_model)

mock_response = None
chat = ['Hi', 'How can I assist']

with patch('web.app.assistant.emotional_journal.openai_chat_request', return_value=[mock_response, None]):
    result = await journal.update_journal(chat)
    print("Result: ", result)
    assert result == (None, None, None)

____________________
--- Journal prompt:
 Conversation:|
Hi
How can I assist| 
--- Journal update response:
 None
____________________
Result:  (None, None, None)


### User profiler tests

In [35]:
from web.app.assistant.user_profile import *

In [36]:
# New profile change
user_profile = '[]'
gpt_model = ''

profile = Profile(user_profile, gpt_model)

mock_response = '{"0" : "Likes this", "1" : "Dislikes this"}'
chat = ['User: Hi', 'Assistant: How can I help']
message = "help me"

with patch('web.app.assistant.user_profile.openai_chat_request', return_value=[mock_response, 'token usage']):
    result = await profile.update_user_profile(previous_messages=chat, user_message=message)
    print("Result: ", result)
    assert result == (['Likes this', 'Dislikes this'], 'token usage')

- Profiler prompts:
 Your task is to update or generate a user profile from recent chat.
Provide concise entries for user preferences, likes, dislikes, and key aspects of the user's personality.
Your response should be short but descriptive, focusing on the most useful data for understanding the user.
- If no changes need to be made to the user profile, your response is empty.
- Don't duplicate the old profile; your response should only include updates.
- To change an entire entry, respond with {"number": "Changed text of this element"}
NEVER RESPOND DIRECTLY TO THE USER!
Follow this JSON format:
{
    "1": "Changed text describing the first element",
    "2": "Changed text describing the second element"
}
 PREVIOUS USER PROFILE:|
{}|
CHAT HISTORY:|
User: Hi
Assistant: How can I help|
Last user message: help me
Remember format: {{"number": "Changed text of this element", "number": "Changed text of this element"}}
- Generated user prifile changes: {"0" : "Likes this", "1" : "Dislikes th

In [37]:
# Update existing profile with new entires
user_profile = '["likes","dislikes"]'
gpt_model = ''

profile = Profile(user_profile, gpt_model)

mock_response = '{"2" : "loves", "3" : "hates"}'
chat = ['User: Hi', 'Assistant: How can I help']
message = "help me"

with patch('web.app.assistant.user_profile.openai_chat_request', return_value=[mock_response, 'token usage']):
    result = await profile.update_user_profile(previous_messages=chat, user_message=message)
    print("Result: ", result)
    assert result == (['likes', 'dislikes', 'loves', 'hates'], 'token usage')

- Profiler prompts:
 Your task is to update or generate a user profile from recent chat.
Provide concise entries for user preferences, likes, dislikes, and key aspects of the user's personality.
Your response should be short but descriptive, focusing on the most useful data for understanding the user.
- If no changes need to be made to the user profile, your response is empty.
- Don't duplicate the old profile; your response should only include updates.
- To change an entire entry, respond with {"number": "Changed text of this element"}
NEVER RESPOND DIRECTLY TO THE USER!
Follow this JSON format:
{
    "1": "Changed text describing the first element",
    "2": "Changed text describing the second element"
}
 PREVIOUS USER PROFILE:|
{"0": "likes", "1": "dislikes"}|
CHAT HISTORY:|
User: Hi
Assistant: How can I help|
Last user message: help me
Remember format: {{"number": "Changed text of this element", "number": "Changed text of this element"}}
- Generated user prifile changes: {"2" : "lo

In [38]:
# Update existing profile with new entires + changing and deleting existing entries
user_profile = '["likes","dislikes"]'
gpt_model = ''

profile = Profile(user_profile, gpt_model)

mock_response = '{"0" : "", "1" : "dislikes likes", "2": "wow new", "4": "cool"}'
chat = ['User: Hi', 'Assistant: How can I help']
message = "help me"

with patch('web.app.assistant.user_profile.openai_chat_request', return_value=[mock_response, 'token usage']):
    result = await profile.update_user_profile(previous_messages=chat, user_message=message)
    print("Result: ", result)
    assert result == (['dislikes likes', 'wow new', 'cool'], 'token usage')

- Profiler prompts:
 Your task is to update or generate a user profile from recent chat.
Provide concise entries for user preferences, likes, dislikes, and key aspects of the user's personality.
Your response should be short but descriptive, focusing on the most useful data for understanding the user.
- If no changes need to be made to the user profile, your response is empty.
- Don't duplicate the old profile; your response should only include updates.
- To change an entire entry, respond with {"number": "Changed text of this element"}
NEVER RESPOND DIRECTLY TO THE USER!
Follow this JSON format:
{
    "1": "Changed text describing the first element",
    "2": "Changed text describing the second element"
}
 PREVIOUS USER PROFILE:|
{"0": "likes", "1": "dislikes"}|
CHAT HISTORY:|
User: Hi
Assistant: How can I help|
Last user message: help me
Remember format: {{"number": "Changed text of this element", "number": "Changed text of this element"}}
- Generated user prifile changes: {"0" : "",

In [39]:
# Error in keys from response
user_profile = '["likes","dislikes"]'
gpt_model = ''

profile = Profile(user_profile, gpt_model)

mock_response = '{"1a" : "232424", "b345" : "dislikes likes", "2": "new"}'
chat = ['User: Hi', 'Assistant: How can I help']
message = "help me"

with patch('web.app.assistant.user_profile.openai_chat_request', return_value=[mock_response, 'token usage']):
    result = await profile.update_user_profile(previous_messages=chat, user_message=message)
    print("Result: ", result)
    assert result == (['likes', 'dislikes', 'new'], 'token usage')

- Profiler prompts:
 Your task is to update or generate a user profile from recent chat.
Provide concise entries for user preferences, likes, dislikes, and key aspects of the user's personality.
Your response should be short but descriptive, focusing on the most useful data for understanding the user.
- If no changes need to be made to the user profile, your response is empty.
- Don't duplicate the old profile; your response should only include updates.
- To change an entire entry, respond with {"number": "Changed text of this element"}
NEVER RESPOND DIRECTLY TO THE USER!
Follow this JSON format:
{
    "1": "Changed text describing the first element",
    "2": "Changed text describing the second element"
}
 PREVIOUS USER PROFILE:|
{"0": "likes", "1": "dislikes"}|
CHAT HISTORY:|
User: Hi
Assistant: How can I help|
Last user message: help me
Remember format: {{"number": "Changed text of this element", "number": "Changed text of this element"}}
- Generated user prifile changes: {"1a" : "2

In [40]:
# Error in response format
user_profile = '["hi"]'
gpt_model = ''

profile = Profile(user_profile, gpt_model)

mock_response = '{"1": ""here is an error", "2": "correct"}'
chat = ['User: Hi', 'Assistant: How can I help']
message = "help me"

with patch('web.app.assistant.user_profile.openai_chat_request', return_value=[mock_response, 'token usage']):
    result = await profile.update_user_profile(previous_messages=chat, user_message=message)
    print("Result: ", result)
    assert result == (None, 'token usage')

- Profiler prompts:
 Your task is to update or generate a user profile from recent chat.
Provide concise entries for user preferences, likes, dislikes, and key aspects of the user's personality.
Your response should be short but descriptive, focusing on the most useful data for understanding the user.
- If no changes need to be made to the user profile, your response is empty.
- Don't duplicate the old profile; your response should only include updates.
- To change an entire entry, respond with {"number": "Changed text of this element"}
NEVER RESPOND DIRECTLY TO THE USER!
Follow this JSON format:
{
    "1": "Changed text describing the first element",
    "2": "Changed text describing the second element"
}
 PREVIOUS USER PROFILE:|
{"0": "hi"}|
CHAT HISTORY:|
User: Hi
Assistant: How can I help|
Last user message: help me
Remember format: {{"number": "Changed text of this element", "number": "Changed text of this element"}}
- Generated user prifile changes: {"1": ""here is an error", "2"

In [41]:
# None response
user_profile = '[]'
gpt_model = ''

profile = Profile(user_profile, gpt_model)

mock_response = None
chat = ['User: Hi', 'Assistant: How can I help']
message = "help me"

with patch('web.app.assistant.user_profile.openai_chat_request', return_value=[mock_response, None]):
    result = await profile.update_user_profile(previous_messages=chat, user_message=message)
    print("Result: ", result)
    assert result == (None, None)

- Profiler prompts:
 Your task is to update or generate a user profile from recent chat.
Provide concise entries for user preferences, likes, dislikes, and key aspects of the user's personality.
Your response should be short but descriptive, focusing on the most useful data for understanding the user.
- If no changes need to be made to the user profile, your response is empty.
- Don't duplicate the old profile; your response should only include updates.
- To change an entire entry, respond with {"number": "Changed text of this element"}
NEVER RESPOND DIRECTLY TO THE USER!
Follow this JSON format:
{
    "1": "Changed text describing the first element",
    "2": "Changed text describing the second element"
}
 PREVIOUS USER PROFILE:|
{}|
CHAT HISTORY:|
User: Hi
Assistant: How can I help|
Last user message: help me
Remember format: {{"number": "Changed text of this element", "number": "Changed text of this element"}}
- Generated user prifile changes: None 
 ________________________________

### Tools tests

In [42]:
from web.app.assistant.tools import *

##### extract_tools

In [43]:
# Valid tools extracted 
tools = Tools(gpt_model='')
user_message = ''
chat_history = ''
token_usage = ''
mock_response = 'news, weather'
with patch('web.app.assistant.tools.openai_chat_request', return_value=[mock_response, token_usage]):
    result = await tools.extract_tools(user_message, chat_history)
    print(result)
    assert result == ['news', 'weather']

mock_response = 'weather'
with patch('web.app.assistant.tools.openai_chat_request', return_value=[mock_response, token_usage]):
    result = await tools.extract_tools(user_message, chat_history)
    print(result)
    assert result == ['weather']


____________________
- Tool extraction:
 NEVER RESPOND USER DIRECTLY!
Previous message: 
User message:  
 Your task is to identify if the user intends to use a tool from the provided list below.
- YOU RESPOND WITH A TOOL ONLY IF YOU ARE 100% SURE USER WNATS TO USE IT.
- If there are no relevant tools to use, your response MUST BE EMPTY.
- Your response is always only a string of tools delimited by a comma. Follow this format:
"tool_name, tool_name, ..."
- NEVER RESPOND DIRECTLY TO THE USER!
- You can only choose tools from this list:
Name: tools_info; Description: Tells about available tools
Name: weather; Description: tells current weather
Name: news; Description: Searches for current top headlines, by key words and categories
Name: calculator; Description: Calculates given expression
 
 news, weather
____________________
['news', 'weather']
____________________
- Tool extraction:
 NEVER RESPOND USER DIRECTLY!
Previous message: 
User message:  
 Your task is to identify if the user in

In [44]:
# Valid and invalid tools
tools = Tools(gpt_model='')
user_message = ''
chat_history = ''

mock_response = 'weatherer, news, calcolator'
with patch('web.app.assistant.tools.openai_chat_request', return_value=[mock_response, token_usage]):
    result = await tools.extract_tools(user_message, chat_history)
    print(result)
    assert result == ['news']
mock_response = 'weather, new, calculator'
with patch('web.app.assistant.tools.openai_chat_request', return_value=[mock_response, token_usage]):
    result = await tools.extract_tools(user_message, chat_history)
    print(result)
    assert result == ['weather','calculator']
    

____________________
- Tool extraction:
 NEVER RESPOND USER DIRECTLY!
Previous message: 
User message:  
 Your task is to identify if the user intends to use a tool from the provided list below.
- YOU RESPOND WITH A TOOL ONLY IF YOU ARE 100% SURE USER WNATS TO USE IT.
- If there are no relevant tools to use, your response MUST BE EMPTY.
- Your response is always only a string of tools delimited by a comma. Follow this format:
"tool_name, tool_name, ..."
- NEVER RESPOND DIRECTLY TO THE USER!
- You can only choose tools from this list:
Name: tools_info; Description: Tells about available tools
Name: weather; Description: tells current weather
Name: news; Description: Searches for current top headlines, by key words and categories
Name: calculator; Description: Calculates given expression
 
 weatherer, news, calcolator
____________________
['news']
____________________
- Tool extraction:
 NEVER RESPOND USER DIRECTLY!
Previous message: 
User message:  
 Your task is to identify if the user

In [45]:
# Errors in response format.
tools = Tools(gpt_model='')
user_message = ''
chat_history = ''

mock_response = 'what ,weather news, calculator, hi, '

with patch('web.app.assistant.tools.openai_chat_request', return_value=[mock_response, token_usage]):
    result = await tools.extract_tools(user_message, chat_history)
    print(result)
    # Only calculator delimeted by comas
    assert result == ['calculator']
    
mock_response = "Sorry I don't see tools here, any questions."
with patch('web.app.assistant.tools.openai_chat_request', return_value=[mock_response, token_usage]):
    result = await tools.extract_tools(user_message, chat_history)
    print(result)
    
    assert result == None

____________________
- Tool extraction:
 NEVER RESPOND USER DIRECTLY!
Previous message: 
User message:  
 Your task is to identify if the user intends to use a tool from the provided list below.
- YOU RESPOND WITH A TOOL ONLY IF YOU ARE 100% SURE USER WNATS TO USE IT.
- If there are no relevant tools to use, your response MUST BE EMPTY.
- Your response is always only a string of tools delimited by a comma. Follow this format:
"tool_name, tool_name, ..."
- NEVER RESPOND DIRECTLY TO THE USER!
- You can only choose tools from this list:
Name: tools_info; Description: Tells about available tools
Name: weather; Description: tells current weather
Name: news; Description: Searches for current top headlines, by key words and categories
Name: calculator; Description: Calculates given expression
 
 what ,weather news, calculator, hi, 
____________________
['calculator']
____________________
- Tool extraction:
 NEVER RESPOND USER DIRECTLY!
Previous message: 
User message:  
 Your task is to ident

In [46]:
# Errors in response format.
tools = Tools(gpt_model='')
user_message = ''
chat_history = ''

mock_response = 'what ,weather news, calculator, hi, '

with patch('web.app.assistant.tools.openai_chat_request', return_value=[mock_response, token_usage]):
    result = await tools.extract_tools(user_message, chat_history)
    print(result)
    # Only calculator delimeted by comas
    assert result == ['calculator']
    
mock_response = "Sorry I don't see tools here, any questions. ,   ,"
with patch('web.app.assistant.tools.openai_chat_request', return_value=[mock_response, token_usage]):
    result = await tools.extract_tools(user_message, chat_history)
    print(result)
    
    assert result == None

____________________
- Tool extraction:
 NEVER RESPOND USER DIRECTLY!
Previous message: 
User message:  
 Your task is to identify if the user intends to use a tool from the provided list below.
- YOU RESPOND WITH A TOOL ONLY IF YOU ARE 100% SURE USER WNATS TO USE IT.
- If there are no relevant tools to use, your response MUST BE EMPTY.
- Your response is always only a string of tools delimited by a comma. Follow this format:
"tool_name, tool_name, ..."
- NEVER RESPOND DIRECTLY TO THE USER!
- You can only choose tools from this list:
Name: tools_info; Description: Tells about available tools
Name: weather; Description: tells current weather
Name: news; Description: Searches for current top headlines, by key words and categories
Name: calculator; Description: Calculates given expression
 
 what ,weather news, calculator, hi, 
____________________
['calculator']
____________________
- Tool extraction:
 NEVER RESPOND USER DIRECTLY!
Previous message: 
User message:  
 Your task is to ident

##### extract_inputs

In [47]:
# Correct inputs in response.
tools = Tools(gpt_model='')
tool = 'weather'
chat_history = ''


mock_response = '''
{
    "location": "Riga"
}'''

with patch('web.app.assistant.tools.openai_chat_request', return_value=[mock_response, token_usage]):
    result  = await tools.extract_inputs(tool, chat_history)
    print(result)

    assert result == ({'location': 'Riga'}, [])
    
tool = 'news'
mock_response = '''
{
    "key_words": "AI",
    "category": "technology"
}'''
with patch('web.app.assistant.tools.openai_chat_request', return_value=[mock_response, token_usage]):
    result = await tools.extract_inputs(tool, chat_history)
    print(result)

    assert result == ({"key_words": "AI", "category": "technology"}, [])

____________________
- Input extraction:
 Function name: weather
- Function description: tells current weather
- Function inputs: ['location']
- Inputs description: Location - city, street or region to check weather
- Conversation: |

Remember the format! Extract inputs for the described function from the conversation and respond with a JSON file containing inputs.
Follow this format:
{
    "input_name1": "input1",
    "input_name2": "input2",
    ...
}
Ensure all inputs are in the same sequence as described.
If an input has '(optional)' in the description, it can be empty.
If there are no inputs in user messages or a required input is missing, your response MUST BE EMPTY.
If there are no inputs, leave the input field EMPTY.
Analyze the entire chat for input extraction. 
{
    "location": "Riga"
}
____________________
({'location': 'Riga'}, [])
____________________
- Input extraction:
 Function name: news
- Function description: Searches for current top headlines, by key words and cate

In [48]:
# Invalid inputs in response.
tools = Tools(gpt_model='')
tool = 'weather'
chat_history = ''


mock_response = '''
{
    "error": "Riga",
    "what": ""
}'''

with patch('web.app.assistant.tools.openai_chat_request', return_value=[mock_response, token_usage]):
    result  = await tools.extract_inputs(tool, chat_history)
    print(result)
    
    assert result == ({'location': None}, ['location'])

____________________
- Input extraction:
 Function name: weather
- Function description: tells current weather
- Function inputs: ['location']
- Inputs description: Location - city, street or region to check weather
- Conversation: |

Remember the format! Extract inputs for the described function from the conversation and respond with a JSON file containing inputs.
Follow this format:
{
    "input_name1": "input1",
    "input_name2": "input2",
    ...
}
Ensure all inputs are in the same sequence as described.
If an input has '(optional)' in the description, it can be empty.
If there are no inputs in user messages or a required input is missing, your response MUST BE EMPTY.
If there are no inputs, leave the input field EMPTY.
Analyze the entire chat for input extraction. 
{
    "error": "Riga",
    "what": ""
}
____________________
Required input not found! - location
({'location': None}, ['location'])


In [49]:
# Error in response format
tools = Tools(gpt_model='')
tool = 'news'
mock_response = '''
How to extract inputs?
{
    "?":"how"?
}'''
with patch('web.app.assistant.tools.openai_chat_request', return_value=[mock_response, token_usage]):
    result = await tools.extract_inputs(tool, chat_history)
    print(result)
    assert result == (None, ['key_words'])

____________________
- Input extraction:
 Function name: news
- Function description: Searches for current top headlines, by key words and categories
- Function inputs: ['key_words', 'category']
- Inputs description: Key words - word or a phrase to search news for. Catogory - you can only select one of these categories: business, entertainment, general, health, science, sports, technology. (optional)
- Conversation: |

Remember the format! Extract inputs for the described function from the conversation and respond with a JSON file containing inputs.
Follow this format:
{
    "input_name1": "input1",
    "input_name2": "input2",
    ...
}
Ensure all inputs are in the same sequence as described.
If an input has '(optional)' in the description, it can be empty.
If there are no inputs in user messages or a required input is missing, your response MUST BE EMPTY.
If there are no inputs, leave the input field EMPTY.
Analyze the entire chat for input extraction. 
How to extract inputs?
{
    "

In [50]:
# Empty on None response.
tools = Tools(gpt_model='')
tool = 'weather'
chat_history = ''


mock_response = ''

with patch('web.app.assistant.tools.openai_chat_request', return_value=[mock_response, token_usage]):
    result  = await tools.extract_inputs(tool, chat_history)
    print(result)
    
    assert result == (None, ['location'])
    
tool = 'news'
mock_response = None
with patch('web.app.assistant.tools.openai_chat_request', return_value=[mock_response, token_usage]):
    result = await tools.extract_inputs(tool, chat_history)
    print(result)
    assert result == (None, ['key_words'])

____________________
- Input extraction:
 Function name: weather
- Function description: tells current weather
- Function inputs: ['location']
- Inputs description: Location - city, street or region to check weather
- Conversation: |

Remember the format! Extract inputs for the described function from the conversation and respond with a JSON file containing inputs.
Follow this format:
{
    "input_name1": "input1",
    "input_name2": "input2",
    ...
}
Ensure all inputs are in the same sequence as described.
If an input has '(optional)' in the description, it can be empty.
If there are no inputs in user messages or a required input is missing, your response MUST BE EMPTY.
If there are no inputs, leave the input field EMPTY.
Analyze the entire chat for input extraction. 
____________________
GPT did not find any inputs!
(None, ['location'])
____________________
- Input extraction:
 Function name: news
- Function description: Searches for current top headlines, by key words and categori

In [51]:
# Only optional input in response.
tools = Tools(gpt_model='')
tool = 'news'
mock_response = '''
{
    "category": "technology"
}
'''
with patch('web.app.assistant.tools.openai_chat_request', return_value=[mock_response, token_usage]):
    result = await tools.extract_inputs(tool, chat_history)
    print(result)
    assert result == ({"key_words": None, "category": "technology"}, ['key_words'])

____________________
- Input extraction:
 Function name: news
- Function description: Searches for current top headlines, by key words and categories
- Function inputs: ['key_words', 'category']
- Inputs description: Key words - word or a phrase to search news for. Catogory - you can only select one of these categories: business, entertainment, general, health, science, sports, technology. (optional)
- Conversation: |

Remember the format! Extract inputs for the described function from the conversation and respond with a JSON file containing inputs.
Follow this format:
{
    "input_name1": "input1",
    "input_name2": "input2",
    ...
}
Ensure all inputs are in the same sequence as described.
If an input has '(optional)' in the description, it can be empty.
If there are no inputs in user messages or a required input is missing, your response MUST BE EMPTY.
If there are no inputs, leave the input field EMPTY.
Analyze the entire chat for input extraction. 
{
    "category": "technology"

##### ask_for_inputs

In [52]:
# Ask for input with optional input passed
tools = Tools(gpt_model='')
tool_name = 'news'
missing_inputs = ['key_words']
inputs = {'category': 'technology'}

tool_result, metadata = tools.ask_for_inputs(tool_name, missing_inputs, inputs)
print(tool_result)
print(json.dumps(metadata, indent=2))


____________________
- Asking for inputs:
Tool result:
 None 
Metadata:
 {'type': 'input_request', 'tool': 'news', 'found_inputs': {'category': 'technology'}, 'missing_inputs': ['key_words'], 'inputs_description': 'Key words - word or a phrase to search news for. Catogory - you can only select one of these categories: business, entertainment, general, health, science, sports, technology. (optional)'}
____________________
None
{
  "type": "input_request",
  "tool": "news",
  "found_inputs": {
    "category": "technology"
  },
  "missing_inputs": [
    "key_words"
  ],
  "inputs_description": "Key words - word or a phrase to search news for. Catogory - you can only select one of these categories: business, entertainment, general, health, science, sports, technology. (optional)"
}


In [53]:
# Ask for input without any input
tools = Tools(gpt_model='')
tool_name = 'news'
missing_inputs = ['key_words']
inputs = None

tool_result, metadata = tools.ask_for_inputs(tool_name, missing_inputs, inputs)
print(tool_result)
print(json.dumps(metadata, indent=2))

____________________
- Asking for inputs:
Tool result:
 None 
Metadata:
 {'type': 'input_request', 'tool': 'news', 'found_inputs': {'key_words': None, 'category': None}, 'missing_inputs': ['key_words'], 'inputs_description': 'Key words - word or a phrase to search news for. Catogory - you can only select one of these categories: business, entertainment, general, health, science, sports, technology. (optional)'}
____________________
None
{
  "type": "input_request",
  "tool": "news",
  "found_inputs": {
    "key_words": null,
    "category": null
  },
  "missing_inputs": [
    "key_words"
  ],
  "inputs_description": "Key words - word or a phrase to search news for. Catogory - you can only select one of these categories: business, entertainment, general, health, science, sports, technology. (optional)"
}


##### run_tools

In [54]:
import web.app.assistant.Tools.tools_functions

In [55]:
# Run tool that doesn't need input.
tools = Tools(gpt_model='')
tools_list = ['tools_info']
inputs = None

result, metadata = await tools.run_tools(tools_list, inputs)

print(result)
print(json.dumps(metadata, indent=2))
assert result == tools_info()
assert metadata ==  {"type": "tool_result", "tool": 'tools_info'}

Here is tools list:
Name: tools_info; Description: Tells about available tools; Inputs: []
Name: weather; Description: tells current weather; Inputs: ['location']
Name: news; Description: Searches for current top headlines, by key words and categories; Inputs: ['key_words', 'category']
Name: calculator; Description: Calculates given expression; Inputs: ['expression']

{
  "type": "tool_result",
  "tool": "tools_info"
}


In [56]:
# Run tools that need input without inputs.
tools = Tools(gpt_model='')
tools_list = ['weather']
inputs = None

result, metadata = await tools.run_tools(tools_list, inputs)

print(result)
print(json.dumps(metadata, indent=2))
assert result == None
assert metadata ==  {'type': 'input_request',
                     'tool': 'weather',
                     'found_inputs': {'location': None},
                     'missing_inputs': ['location'],
                     'inputs_description': 'Location - city, street or region to check weather'}

tools_list = ['news']
inputs = None
result, metadata = await tools.run_tools(tools_list, inputs)

print(result)
print(json.dumps(metadata, indent=2))
assert result == None
assert metadata == {"type": "input_request",
                    "tool": "news",
                    "found_inputs": {"key_words": None ,"category": None},
                    "missing_inputs": ["key_words", "category"],
                    "inputs_description": "Key words - word or a phrase to search news for. Catogory - you can only select one of these categories: business, entertainment, general, health, science, sports, technology. (optional)"}

____________________
- Asking for inputs:
Tool result:
 None 
Metadata:
 {'type': 'input_request', 'tool': 'weather', 'found_inputs': {'location': None}, 'missing_inputs': ['location'], 'inputs_description': 'Location - city, street or region to check weather'}
____________________
None
{
  "type": "input_request",
  "tool": "weather",
  "found_inputs": {
    "location": null
  },
  "missing_inputs": [
    "location"
  ],
  "inputs_description": "Location - city, street or region to check weather"
}
____________________
- Asking for inputs:
Tool result:
 None 
Metadata:
 {'type': 'input_request', 'tool': 'news', 'found_inputs': {'key_words': None, 'category': None}, 'missing_inputs': ['key_words', 'category'], 'inputs_description': 'Key words - word or a phrase to search news for. Catogory - you can only select one of these categories: business, entertainment, general, health, science, sports, technology. (optional)'}
____________________
None
{
  "type": "input_request",
  "tool": "ne

In [57]:
# Run tools that need input with correct inputs.
tools = Tools(gpt_model='')
tools_list = ['weather']
inputs = {"location": 'Riga'}

result, metadata = await tools.run_tools(tools_list, inputs)

print(result)
print(json.dumps(metadata, indent=2))
assert result == weather(inputs['location'])
assert metadata ==  {
  "type": "tool_result",
  "tool": "weather",
  "found_inputs": {
    "location": "Riga"
  }
}

tools_list = ['news']
inputs = {'key_words': 'AI'}
result, metadata = await tools.run_tools(tools_list, inputs)

print(result)
print(json.dumps(metadata, indent=2))
assert result == news(inputs["key_words"])
assert metadata == {
  "type": "tool_result",
  "tool": "news",
  "found_inputs": {
    "key_words": "AI"
  }
}

Weather in: Riga
Status: fog
Temperature: 1.71°C
Humidity: 98%
Wind Speed: 1.54 m/s
{
  "type": "tool_result",
  "tool": "weather",
  "found_inputs": {
    "location": "Riga"
  }
}
Tell user about these found articles. Mention source and link to an article.[{'source': {'id': None, 'name': 'Yahoo Entertainment'}, 'author': 'Frank Schwab', 'title': 'Tom Brady officially a part owner of the Raiders after NFL owners approve it - Yahoo Sports', 'description': 'The former Patriots and Buccaneers QB is now part of NFL ownership.', 'url': 'https://sports.yahoo.com/tom-brady-officially-a-part-owner-of-the-raiders-after-nfl-owners-approve-it-201813931.html', 'urlToImage': 'https://s.yimg.com/ny/api/res/1.2/trvAomImzZTwGEGXJxdRkg--/YXBwaWQ9aGlnaGxhbmRlcjt3PTEyMDA7aD04MDA-/https://s.yimg.com/os/creatr-uploaded-images/2024-10/1d64fc80-8747-11ef-9ff8-dad51ec19a62', 'publishedAt': '2024-10-15T22:19:30Z', 'content': 'Fox Sports commentator and former NFL quarterback Tom Brady is officially a part owne

In [58]:
# Test tool that throws exeption
tools = Tools(gpt_model='', tools_path='web/app/assistant/Tools/test_tool.txt')
tools_list = ['test_tool']
inputs = {'exeption': True}

mock_response, token_usage = 'Test tool exeption description', ''

with patch('web.app.assistant.tools.openai_chat_request', return_value=[mock_response, token_usage]):
    result, metadata = await tools.run_tools(tools_list, inputs)

print(result)
print(json.dumps(metadata, indent=2))

assert result == mock_response
assert metadata ==  {
    "type": "input_request",
    "tool": "test_tool",
    "found_inputs": {"exeption": True},
    "missing_inputs": []
}

Recived exception running tool:
 Test Exception 
 Inputs:
 {'exeption': True}
Test tool exeption description
{
  "type": "input_request",
  "tool": "test_tool",
  "found_inputs": {
    "exeption": true
  },
  "missing_inputs": []
}


In [59]:
a = 's'
if a:
    print('n')

n


##### handle_tools

In [74]:
# Tool extractor did not find any tools
tools = Tools(gpt_model='')
user_message = ''
chat_history = ['hi', 'hello']
extract_inputs = False
messages_for_input_extraction = 3
token_usage = ''
mock_response = None
with patch('web.app.assistant.tools.Tools.extract_tools', return_value=mock_response):
    result = await tools.handle_tools(extract_inputs, user_message, chat_history, messages_for_input_extraction)
    print(result)
    assert result == (None, None)

No tools detected!
(None, None)


In [79]:
# Tool extractor did not find any tools
tools = Tools(gpt_model='')
user_message = ''
chat_history = ['hi', 'hello']
extract_inputs = False
messages_for_input_extraction = 3
token_usage = ''
mock_response = ['news']
with patch('web.app.assistant.tools.Tools.extract_tools', return_value=mock_response):
    result = await tools.handle_tools(extract_inputs, user_message, chat_history, messages_for_input_extraction)
    print(result)
    assert result == (None, {
        'type': 'input_request',
        'tool': 'news',
        'found_inputs': {
            'key_words': None,                      
            'category': None
            },
        'missing_inputs': ['key_words'],
        'inputs_description': 'Key words - word or a phrase to search news for. Catogory - you can only select one of these categories: business, entertainment, general, health, science, sports, technology. (optional)'
    })

____________________
- Asking for inputs:
Tool result:
 None 
Metadata:
 {'type': 'input_request', 'tool': 'news', 'found_inputs': {'key_words': None, 'category': None}, 'missing_inputs': ['key_words'], 'inputs_description': 'Key words - word or a phrase to search news for. Catogory - you can only select one of these categories: business, entertainment, general, health, science, sports, technology. (optional)'}
____________________
(None, {'type': 'input_request', 'tool': 'news', 'found_inputs': {'key_words': None, 'category': None}, 'missing_inputs': ['key_words'], 'inputs_description': 'Key words - word or a phrase to search news for. Catogory - you can only select one of these categories: business, entertainment, general, health, science, sports, technology. (optional)'})


In [ ]:
# Tool extractor found tool but no input extraction
tools = Tools(gpt_model='')
user_message = ''
chat_history = ['hi', 'hello']
extract_inputs = False
messages_for_input_extraction = 3
token_usage = ''
mock_response = None
with patch('web.app.assistant.tools.Tools.extract_tools', return_value=mock_response):
    result = await tools.handle_tools(extract_inputs, user_message, chat_history, messages_for_input_extraction)
    print(result)
    assert result == (None, None)

### Recommender

In [11]:
from web.app.assistant.recommender import *


In [14]:
RECOMMENDER_DEBUG = True

In [15]:
rec_tree = RecommerdationTree(3)


rec_tree.build_tree([Data([1,2,3], 1)])

In [16]:
RECOMMENDER_DEBUG

True

### Responder

In [4]:
from web.app.assistant.responder import *

In [31]:
settings = AssistantSettings(
    responder_gpt_model='gpt-4o-mini',
    profiler_gpt_model='gpt-4o-mini',
    messages_for_profile_update=1,
    messages_till_profile_update=2,
    messages_for_input_extraction=3,
    messages_for_journal_update=1,
    messages_till_journal_update=2,
    responder_personality='')
responder = Responder(
    user_profile='[]',
    chat_history=['Assistant: How can I help?'],
    assistant_settings=settings,
    message_count=1,
    emotional_journal=EmotionalJournal(
        journal='{}', updates_count=0, gpt_model='gpt-4o-mini'))

In [32]:
response, meta, profile, tokens = await responder.handle_user_message(
    user_message='Hi',
    use_tools=True,
    extract_inputs=True)


____________________
- Tool extraction:
 NEVER RESPOND USER DIRECTLY!
Previous message: Assistant: How can I help?
User message: Hi 
 Your task is to identify if the user intends to use a tool from the provided list below.
- YOU RESPOND WITH A TOOL ONLY IF YOU ARE 100% SURE USER WNATS TO USE IT.
- If there are no relevant tools to use, your response MUST BE EMPTY.
- Your response is always only a string of tools delimited by a comma. Follow this format:
"tool_name, tool_name, ..."
- NEVER RESPOND DIRECTLY TO THE USER!
- You can only choose tools from this list:
Name: tools_info; Description: Tells about available tools
Name: weather; Description: tells current weather
Name: news; Description: Searches for current top headlines, by key words and categories
Name: calculator; Description: Calculates given expression
 
 
____________________
GPT tied to extract tools but found no tools to use
No tools detected!
- _Responder_ Sysytem prompt:
You are a helpful wellbeing assistant.
You care a

# Tests

In [60]:
tokens_used = {'Tools': {'prompt_tokens': 508, 'completion_tokens': 18, 'total_tokens': 526},
'Profiler': {
  "prompt_tokens": 440,
  "completion_tokens": 16,
  "total_tokens": 456,
  "prompt_tokens_details": {
    "cached_tokens": 0
  },
  "completion_tokens_details": {
    "reasoning_tokens": 0
  }
}, 'Journal': {
  "prompt_tokens": 139,
  "completion_tokens": 25,
  "total_tokens": 164,
  "prompt_tokens_details": {
    "cached_tokens": 0
  },
  "completion_tokens_details": {
    "reasoning_tokens": 0
  }
}, 'Responder': {}}
for module in tokens_used:
    if len(tokens_used[module]) != 0:
        if tokens_used[module]['total_tokens'] != 0:
            if module == 'Tools':
                print(module)
                print(tokens_used[module]['prompt_tokens'])

Tools
508
